In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import gc
import random
import seaborn as sns
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix    
import matplotlib.pyplot as plt 
from catboost import CatBoostClassifier, cv, Pool
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
np.random.seed(64)
random.seed(64)

In [ ]:
data_root = os.environ.get('KAGGLE_DIR', '../input')
train = pd.read_parquet(f'{data_root}/tpsdec2021parquet/train.pq')
#pseudo = pd.read_csv(f'{data_root}/tps12-pseudolabels/tps12-pseudolabels_v2.csv', dtype=train.dtypes.to_dict())
test = pd.read_parquet(f'{data_root}/tpsdec2021parquet/test.pq')

In [ ]:
train.head()

In [ ]:
train.describe().T

In [ ]:
test.head()

In [ ]:
#pseudo.head()

In [ ]:
train['Cover_Type'].value_counts()

In [ ]:
all_df = pd.concat([train.assign(ds=0), test.assign(ds=2)]).reset_index(drop=True).drop(
    columns=['Soil_Type7', 'Soil_Type15'] # 0 variance
)

In [ ]:
train = train.drop(columns=['Soil_Type7', 'Soil_Type15'])
test = test.drop(columns=['Soil_Type7', 'Soil_Type15'])
train = train.astype({'Cover_Type': np.int8})


In [ ]:
# types = {'Cover_Type': np.int8}
# train = all_df.loc[all_df.ds == 0].astype(types).drop(columns=['ds'])
# #pseudo = all_df.loc[all_df.ds == 1].astype(types).drop(columns=['ds'])
# test = all_df.loc[all_df.ds == 2].drop(columns=['Cover_Type', 'ds'])

In [ ]:
label_encoder = LabelEncoder()

train = train.loc[train.Cover_Type != 5] # Excluding label 5 because there is only 1 training example

X = train.drop(columns=['Id', 'Cover_Type']).astype(np.float32).to_numpy()
y = label_encoder.fit_transform(train.Cover_Type)
feat_names = train.columns.drop(['Id', 'Cover_Type'])
colum_names = train.columns
del train
num_class = len(label_encoder.classes_)

# X_pseudo = pseudo.drop(columns=['Id', 'Cover_Type']).astype(np.float32).to_numpy()
# y_pseudo = label_encoder.transform(pseudo.Cover_Type)
# del pseudo

X_test = test.drop(columns=['Id']).astype(np.float32).to_numpy()
del test


XGB_col_names = [f'{cls}' for cls in label_encoder.classes_] # Column names will be used to save results 
CAT_col_names = [f'{cls}' for cls in label_encoder.classes_]

Softprob classifier: To output probabilities instead of classes

Subsample is the fraction of the samples that will be used at each iteration of the booster.

lambda is l2 regularization parameter. 

For more detailed explanation - do refer to https://www.kaggle.com/kaaveland/tps202112-reasonable-xgboost-model



In [ ]:
params = {
    'num_class': num_class,
    'objective': 'multi:softprob',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': ['merror', 'mlogloss'],
    'learning_rate': .05,
    'max_depth': 0,
    'subsample': .20,
    'sampling_method': 'gradient_based',
    'seed': 64,
    'grow_policy': 'lossguide',
    'max_leaves': 255,
    'lambda': 75,
}

In [ ]:
%%time
XGB_val_proba = np.zeros((len(y), num_class), dtype=np.float32)
XGB_test_proba = np.zeros((len(X_test), num_class), dtype=np.float32)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=64)
for train_idx, val_idx in cv.split(X, y):
    gc.collect()
    dval = xgboost.DMatrix(X[val_idx], label=y[val_idx])
    booster: xgboost.Booster = xgboost.train(
        params=params,
        dtrain= xgboost.DMatrix(X[train_idx], label=y[train_idx]),
        num_boost_round= 1500,
        evals=[(dval, 'val')],
        early_stopping_rounds=100,
        verbose_eval=100,
    )
    XGB_val_proba[val_idx] = booster.predict(
        dval, iteration_range=(0, booster.best_iteration + 1)
    )
    XGB_test_proba += booster.predict(
        xgboost.DMatrix(X_test), iteration_range=(0, booster.best_iteration + 1)
    ) / cv.n_splits

In [ ]:
y_pred = label_encoder.inverse_transform(XGB_val_proba.argmax(axis=1))
y_true = label_encoder.inverse_transform(y)

print(classification_report(y_true, y_pred))

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize': (18, 9), 'figure.frameon': False})

fscore = booster.get_fscore()
fscore = pd.DataFrame({'value': fscore.values()}, index=feat_names[np.arange(len(fscore))])
fscore.plot.barh();

In [ ]:
XGB_test_preds = pd.DataFrame(XGB_test_proba, columns=XGB_col_names)

In [ ]:
#y_pred = label_encoder.inverse_transform(test_proba.argmax(axis=1))
#sub = pd.read_parquet(f'{data_root}/tpsdec2021parquet/test.pq', columns=['Id']).assign(Cover_Type=y_pred)
#sub.head()
#sub.to_csv('submission.csv', index=False)
#sub.shape

In [ ]:
catboost = CatBoostClassifier(use_best_model = True, random_seed = 64,eval_metric='AUC',iterations=1500)

In [ ]:
%%time
CAT_test_proba = np.zeros((len(X_test), num_class), dtype=np.float32)
CAT_val_proba = np.zeros((len(y), num_class), dtype=np.float32)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=64)
for train_idx, val_idx in cv.split(X, y):
    gc.collect()
    catboost.fit(X[train_idx], y[train_idx], eval_set= (X[val_idx], y[val_idx]), early_stopping_rounds=100,verbose_eval = 100)
    XGB_val_proba[val_idx] = catboost.predict_proba(X[val_idx])  
    CAT_prediction = catboost.predict_proba(X_test)
    CAT_test_proba += CAT_prediction/5

In [ ]:
#y_pred = label_encoder.inverse_transform(val_proba.argmax(axis=1))
#y_true = label_encoder.inverse_transform(y)

#print(classification_report(y_true, y_pred))

In [ ]:
CAT_test_preds = pd.DataFrame(CAT_test_proba, columns=CAT_col_names)

In [ ]:
final_results = pd.DataFrame(columns = ['XGB_Output', 'XGB_Prob', 'CAT_Output', 'CAT_Prob', 'Final_Pred'])

final_results['XGB_Output'] = XGB_test_preds.idxmax(axis =1)
final_results['XGB_Prob'] = XGB_test_preds.max(axis =1)
final_results['CAT_Output'] = CAT_test_preds.idxmax(axis =1)
final_results['CAT_Prob'] = CAT_test_preds.max(axis =1)

In [ ]:
check = final_results[final_results['XGB_Output'] != final_results['CAT_Output']]

In [ ]:
check1 = check[check['XGB_Prob']<check['CAT_Prob']]

In [ ]:
check1.head(25)

In [ ]:
XGB_test_preds.head(50)

In [ ]:
def returnFinalCover(row):
    if row[1] > row[3]:
        return row[0]
    else:
        return row [2]  

In [ ]:
final_results['Final_Pred'] = final_results.apply(returnFinalCover,axis =1)

In [ ]:
final_results.head(25)

In [ ]:
y_pred = final_results['Final_Pred'].astype(np.int8)
sub = pd.read_parquet(f'{data_root}/tpsdec2021parquet/test.pq', columns=['Id']).assign(Cover_Type=y_pred)
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)